In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
from copulas.multivariate import GaussianMultivariate
import numpy as np
from itertools import combinations
import importlib
import logging
import itertools



In [3]:
import Mobility_simulator
importlib.reload(Mobility_simulator)

DEBUG: Filtering where year=2023
DEBUG: Counts of Individual Car Journeys by travel day: IndividualID  TravDay
2022008400    6          2
              7          4
2022008401    1          4
              6          4
2022008405    1          4
              2          3
              7          2
2022008406    1          5
              6          2
2022008491    2          3
Name: JourSeq, dtype: int64
INFO: As we can see more than 10 car journeys a day are infrequent. So we shall remove these
DEBUG: Creating a matrix with first column individual ID and remaining columns corresponding to num of joourneys for each travel day
DEBUG: First 5 rows of travel matrix [[2022008400          0          0          0          0          0
           2          4]
 [2022008401          4          0          0          0          0
           4          0]
 [2022008405          4          3          0          0          0
           0          2]
 [2022008406          5          0          0    

<module 'Mobility_simulator' from '/home/trapfishscott/Cambridge24.25/Energy_thesis/Mobility_simulator.py'>

In [2]:
Data_folder = os.getcwd() + "/Data"

Data_folder

'/home/trapfishscott/Cambridge24.25/Energy_thesis/Data'

In [4]:
## Import pickls

car_df = pd.read_pickle(Data_folder + "/df_car.pkl")

In [5]:
car_df

,DayID,IndividualID,TravDay,JourSeq,TripPurpFrom_B01ID,TripPurpTo_B01ID,TripStart,TripEnd,TripDisExSW,TravelYear,TravelWeekDay_B03ID
0,2014050554,2014007801,7,1,3,1,420.0,425.0,2.0,2014,1
1,2014050554,2014007801,7,3,3,2,1080.0,1125.0,10.0,2014,1
2,2014050561,2014007802,7,1,3,1,360.0,395.0,13.0,2014,1
3,2014050561,2014007802,7,3,3,2,1200.0,1220.0,5.0,2014,1
4,2014050567,2014007803,6,2,2,3,1140.0,1155.0,3.5,2014,1
...,...,...,...,...,...,...,...,...,...,...,...
5342915,2023101345,2023017037,6,1,3,2,565.0,580.0,9.3,2024,2
5342916,2023101345,2023017037,6,2,2,2,590.0,610.0,7.0,2024,2
5342917,2023101345,2023017037,6,3,2,3,620.0,655.0,16.0,2024,2
5342918,2023101346,2023017037,7,1,3,2,710.0,750.0,20.0,2024,2
